# pytorch 설치하기

In [ ]:
# (설치 방법) https://pytorch.org/get-started/locally/
# pip3
# !pip3 install torch torchvision

In [ ]:
import torch
x = torch.rand(5,3)
print(x)

# 빠른 시작

In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda, Compose
import matplotlib.pyplot as plt

PyTorch는 **TorchText**, **TorchVision** 및 **TorchAudio**와 같이 특화된 라이브러리 데이터셋을 함께 제공한다.   
<u>이 튜토리얼에서는 **TorchVision** 데이터 셋을 사용한다.</u>

```torchvision.dataset``` 모듈은 실제 비전 데이터에 대한 data을 포함하며, 이 튜토리얼에서는 FashionMNIST 데이터셋을 사용한다.  
모든 **TorchVision** dataset은 샘플과 정답을 변경하기 위한 ```transform```과 ```target_transform``` 두 인자를 포함한다.

In [ ]:
# 공개 데이터셋에서 학습 데이터 다운
training_data = datasets.FashionMNIST(root = 'data', train = True, download = True, transform = ToTensor(),)

# 공개 데이터셋에서 테스트 데이터 다운
test_data = datasets.FashionMNIST(root = 'data', train = False, download = True, transform = ToTensor(),)

```Dataset```을 ```DataLoader```의 인자로 전달한다. 이는 데이터셋을 순회 가능한 객체(iterable)로 감싸고, 자동화된 배치(batch), 샘플링(sampling), 섞기(shuffle) 및 다중 프로세스로 데이터 불러오기 (multiprocess data loading)를 지원한다.  
여기서 배치 크기(batch size)는 64로 정의한다. 즉, 데이터로더(dataloader) 객체의 각 요소는 64개의 특징(feature)과 정답(label)을 묶음(batch)으로 반환한다.

In [ ]:
batch_size = 64

# 데이터로더 생성
train_dataloader = DataLoader(training_data, batch_size = batch_size)
test_dataloader = DataLoader(test_data, batch_size = batch_size)

for X, y in test_dataloader:
    print(f"Shape of X[N, C, H, W] : {X.shape}")
    print(f"Shape of y : {y.shape, y.dtype}")
    break

   > [DATASET과 DATALOADER(데이터 불러오기)](https://tutorials.pytorch.kr/beginner/basics/data_tutorial.html)

# 모델 만들기
---
PyTorch에서 신경망 모델은 ```nn.Module```을 상속받는 클래스(class)를 생성해 정의한다.  
* ```__init__``` 함수에서 신경망 계층(layer) 정의.
* ```forward``` 함수에서 신경망 데이터를 어떻게 전달할지 지정.

가능한 경우 GPU로 신경망을 이동시켜 연산을 가속(accelearte)한다.

In [ ]:
# 학습에 사용할 CPU or GPU 장치 획득
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"using {device} device")

# 모델 정의
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )
        
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits
    
model = NeuralNetwork().to(device)
print(model)

> [PyTorch에서 신경망 정의하는 방법](https://tutorials.pytorch.kr/beginner/basics/buildmodel_tutorial.html)

# 모델 매개변수 최적화
---
모델을 학습하려면 손실 함수(loss function)와 옵티마이저(optimizer)가 필요하다.

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr = 1e-3)

각 학습 단계(training loop)에서 모델은 학습 데이터셋에 대한 예측 수행 및 예측 오류를 역전파해 모델의 매개변수를 조정한다.

In [ ]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        
        # 예측 오류 계산
        pred = model(X)
        loss = loss_fn(pred, y)
        
        # 역전파
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss : {loss :>7f} [{current :>5d}/{size : >5d}]")

모델이 학습하고 있는지 확인하기 위해 테스트 데이터셋으로 모델 성능 확인

In [ ]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0,0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

학습은 여러번의 반복(epochs)을 거쳐 수행된다.  
각 반복에서 모델은 더 나은 예측을 위해 매개변수를 학습하고 반복별 모델의 정확도(accuracy)와 손실(loss)을 출력한다. 

In [ ]:
epochs = 5

for t in range(epochs):
    print(f"Epoch {t+1}\n",'-'*30)
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Complete")

> [모델 학습방법](https://tutorials.pytorch.kr/beginner/basics/optimization_tutorial.html)

# 모델 저장하기
---
모델을 저장하는 일반적인 방법은 (*모델의 매개변수 포함) 내부 상태 사전(iternal state dictionary)을 직렬화(serialize)화는 것이다.

In [ ]:
torch.save(model.state_dict(), 'model.pth')
print('Saved PyTorch Model State to model.pth')

# 모델 불러오기
---
모델을 불러오는 과정에는 모델 구조를 다시 만들고 상태 사전을 모델에 불러오는 과정이 포함된다.

In [ ]:
model = NeuralNetwork()
model.load_state_dict(torch.load("model.pth"))

이제 위 모델을 사용해 예측할 수 있다.

In [ ]:
classes = [
    'T-shirt/top',
    'Trouser',
    'Pullover',
    'Dress',
    'coat',
    'Sandal',
    'Shirt',
    'Sneaker',
    'Bag',
    'Ankle boot',
]

model.eval()
x, y = test_data[0][0], test_data[0][1]

with torch.no_grad():
    pred = model(x)
    pred = model(x)
    
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f"predicted : {predicted}, Actual : {actual}")
# with torch.no_grad():
#     pred = model(x)
#     predicted, actual = classes[pred[0].argmax(0)], classes[y]
#     print(f"Predicted : {predicted}, Actual : {actual} ")

> [모델 저장하고 불러오기](https://tutorials.pytorch.kr/beginner/basics/saveloadrun_tutorial.html)